# Example pipeline
Author: Jens Petit

In this notebook the whole process for a creating a single map like graph for two lines is outlayed and the results in the intermediate steps are visible.

A connection to the database is necessary to load the data.

In [1]:
from graph_utils import pose_graph_nx as pg
from graph_utils import plotter as plt
from graph_utils import optimizer_matrix_sparse as opti
from graph_utils.db_interface import DBInterface as dbi
from graph_utils import representor
from graph_utils import sparsifier
from graph_utils import pose_graph_helper
from graph_utils import merger
import networkx as nx
import pdb

import matplotlib.pyplot as pyplt
import pickle
import math
import numpy as np
import pdb 
import pandas as pd
from graph_utils import pose_graph_helper as pg_help

import copy
%load_ext autoreload
%autoreload 2

## Loading the raw data and merging them into a single graph object
The graphs will be written to a single pose graph object called base_graph.

In this example we make the map for the lines 54 and 155 which share a part of the road network.

In [2]:
buslines = [54, 155]
lines_route = [(x, '%') for x in buslines]

start_date = '2018-02-15'
dates = pose_graph_helper.createDays(start_date, 1)
print("Lines {} on dates {}".format(lines_route, dates))

graphs = pose_graph_helper.getGraphsDateLine(dates, lines_route, limit_line=200)

print("We have {} graphs in this map.".format(len(graphs)))
print("Start merging them")

base_graph = graphs.pop(0)

for i, graph in enumerate(graphs):
    if (i % 20 == 0):
        print("Merged {} out of {}".format(i, len(graphs)))
    try:
        merger.globalXYMerge(base_graph, graph)
    except nx.exception.NetworkXError:
        print("There are duplicate ids, lets jump this graph...")
        pass

Lines [(54, '%'), (155, '%')] on dates ['2018-02-15']
Starting to get trip Ids for 1 days and 2 lines
Now on date 2018-02-15.
Starting to get trip IDs. Takes a while....
We have 368 graphs
No data for this trip B:54:701!
No data for this trip B:54:16!
No data for this trip B:54:1!
No data for this trip B:54:13!
No data for this trip B:54:2!
No data for this trip 1681612461!
No data for this trip 1681612899!
No data for this trip 1681612904!
No data for this trip 1681636169!
No data for this trip 1681636600!
No data for this trip B:54:12!
No data for this trip B:54:22!
No data for this trip B:54:3!
No data for this trip 1681636930!
No data for this trip B:54:724!
No data for this trip B:54:749!
No data for this trip B:54:7!
No data for this trip 1681651998!
No data for this trip B:54:21!
No data for this trip 1681652423!
No data for this trip 1681652860!
No data for this trip 1681653722!
No data for this trip 1681654568!
No data for this trip B:54:719!
No data for this trip B:54:4!
No d

In [3]:
plotter = plt.Plotter()
heatmap_raw = plotter.addHeatmapLayer([base_graph])
heatmap_raw.max_intensity = 20
plotter.plotGmaps()

Figure(layout=FigureLayout(height='420px'))

## Applying the window smoother

In [4]:
after_window = pose_graph_helper.windowSmoother(base_graph, radius=15, angle=1)

In window smoother for 98602 nodes. Might take a
          while...
Finished creating tree....
Finished finding NN
on node 0 of 98602
on node 10000 of 98602
on node 20000 of 98602
on node 30000 of 98602
on node 40000 of 98602
on node 50000 of 98602
on node 60000 of 98602
on node 70000 of 98602
on node 80000 of 98602
on node 90000 of 98602


In [5]:
plotter = plt.Plotter()
heatmap_after = plotter.addHeatmapLayer([after_window])
heatmap_after.max_intensity = 20
plotter.plotGmaps()

Figure(layout=FigureLayout(height='420px'))

## Finding representative centroids

In [6]:
represented = representor.Representor(after_window)
represented.computeCentroids(radius=15, angle=1, cutoff_percent=10)

0 of 98602 are represented
800 of 98602 are represented
1700 of 98602 are represented
4500 of 98602 are represented
6700 of 98602 are represented
12300 of 98602 are represented
19300 of 98602 are represented
20800 of 98602 are represented
22900 of 98602 are represented
27500 of 98602 are represented
39800 of 98602 are represented
43900 of 98602 are represented
45500 of 98602 are represented
45700 of 98602 are represented
46700 of 98602 are represented
49900 of 98602 are represented
50100 of 98602 are represented
51900 of 98602 are represented
52500 of 98602 are represented
60900 of 98602 are represented
64600 of 98602 are represented
67500 of 98602 are represented
71300 of 98602 are represented
75300 of 98602 are represented
84400 of 98602 are represented
88900 of 98602 are represented
89200 of 98602 are represented
93500 of 98602 are represented
95700 of 98602 are represented
96200 of 98602 are represented
97400 of 98602 are represented
97900 of 98602 are represented
98100 of 98602 ar

## Drawing edges

In [7]:
final_map = represented.exportCentroidsInGraph()

The edge cutoff is 2 representing less than 2%
The edge cutoff is 26 representing less than 5%


### Intermediate graph

The intermediate graph consists of all edges which connect a vehicle trajectory on the centroids. Therefore it contains many edges which get sparsified trhough the search on each of these edges.

In [8]:
intermediate_graph = represented.intermediate_graph

In [9]:
position = np.array([ 48.165152, 11.588042 ])
subgraph = intermediate_graph.exportSubgraphNN(lat_lon=position, k=800)
plotter = plt.Plotter()
plotter.addNodeLayer(subgraph)
plotter.addEdgeLayer(subgraph)
plotter.plotGmaps()

Not enough a nearest neighbors within threshold!
Doubling threshold...
Not enough a nearest neighbors within threshold!
Doubling threshold...
Not enough a nearest neighbors within threshold!
Doubling threshold...


Figure(layout=FigureLayout(height='420px'))

### Final graph

In [10]:
position = np.array([ 48.165152, 11.588042 ])
subgraph = final_map.exportSubgraphNN(lat_lon=position, k=800)
plotter = plt.Plotter()
plotter.addNodeLayer(subgraph)
plotter.addEdgeLayer(subgraph)
plotter.plotGmaps()

Not enough a nearest neighbors within threshold!
Doubling threshold...
Not enough a nearest neighbors within threshold!
Doubling threshold...
Not enough a nearest neighbors within threshold!
Doubling threshold...


Figure(layout=FigureLayout(height='420px'))

## Sparsifying the graph
Sparsification (dropping nodes on linear segments) can be done with the Ramer-Douglas-Peucker algorithm.

In [20]:
print(len(final_map.nx_graph.nodes))
print(len(final_map.nx_graph.edges))

splitters = sparsifier.findSetSplitters(final_map.nx_graph)
print(len(splitters))

1888
1987
270


In [21]:
sparse_full_map = sparsifier.sparsifyPoseGraph(final_map, criteria="rdp", epsilon=2)

print(len(sparse_full_map.nx_graph.nodes))
print(len(sparse_full_map.nx_graph.edges))

Dropped 1170 edges.
718
766


In [13]:
plotter = plt.Plotter()
plotter.addNodeLayer(sparse_full_map)
plotter.addEdgeLayer(sparse_full_map)
plotter.plotGmaps()

Figure(layout=FigureLayout(height='420px'))

## Creating the abstract graph

In [16]:
abstract_map = sparsifier.sparsifyPoseGraph(final_map, criteria="abstract")

print(len(abstract_map.nx_graph.nodes))

Dropped 1618 edges.
270


In [17]:
plotter = plt.Plotter()
plotter.addNodeLayer(abstract_map)
plotter.addEdgeLayer(abstract_map)
plotter.addDoorLayer(abstract_map)
plotter.plotGmaps()


Figure(layout=FigureLayout(height='420px'))